# Analysis of Strang-carryover with RK3/RK4 for AdImEx Advection
## Plan
- Discretise the advection equation in space using quasi-cubic
- Aapply Strang-carryover adaptively implicitly (AdImEx) with high-order corrections treated explicitly.
- Analyse the stabiltiy of the Strang-carryover scheme

## Discretisation of the Advection Equation
\begin{eqnarray}
\frac{\partial \psi}{\partial t} &=& -u \frac{\partial \psi}{\partial x} \\
\text{Assume } \psi_j &=& e^{ikj\Delta x},\ c = u\Delta t/\Delta x,\ \lambda = u/\Delta x \\
\end{eqnarray}
\begin{eqnarray}
\text{Analytic } \frac{dy}{dt} &=& - i\lambda k\Delta x y \\
\end{eqnarray}
The spatially discretised advection equation can be written as a correction on upwind:
\begin{eqnarray}
\frac{dy}{dt} &=& -\lambda(\mu  + \eta) y\\
\text{where } \mu &=& 1-\cos k\Delta x + i\sin k\Delta x \ \text{ (upwind)}\\
\text{and }   \eta &=& -\mu + \left\{
\sum_\ell w_\ell\  e^{i\ell k\Delta x} - \sum_{\ell-1} w_\ell\  e^{i\ell k\Delta x}
\right\}  \ \text{ (correction)}
\end{eqnarray}

The correction weights for a more general, higher-order discretisation of $\frac{\partial \psi}{\partial x}$:
\begin{eqnarray}
\frac{\partial \psi}{\partial x}_{HOj} &=& \frac{\psi_{j+1/2} - \psi_{j-1/2}}{\Delta x} \\
\text{where } \psi_{j+1/2} &=& \sum_{j+\ell} w_\ell\ \psi_{j+\ell}\\
\implies
\frac{\partial \psi}{\partial x}_{HOj} &=& \sum_{j+\ell} \frac{w_\ell\ \psi_{j+\ell}}{\Delta x}
                                    - \sum_{j+\ell-1} \frac{w_\ell\ \psi_{j+\ell}}{\Delta x}\\
\implies -u\frac{\partial \psi}{\partial x}_{HOj} &=& -\psi_j \lambda \left\{
\sum_\ell w_\ell\  e^{i\ell k\Delta x} - \sum_{\ell-1} w_\ell\  e^{i\ell k\Delta x}
\right\}
\end{eqnarray}


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
#import fluidfoam as foam
from sympy import latex
from sympy import I, E, pi, Q

In [ ]:
# mu and eta for a scheme with given weights
kDx = sy.symbols("kΔx", real=True)
def muEta(indicies, weights):
    mu = 1 - E**(-I*kDx)
    eta = -mu
    for j,w in zip(indicies, weights):
        eta += w*(E**(j*I*kDx) - E**((j-1)*I*kDx))
    return mu, eta

# Quasi-cubic
indicies3 = [-1,0,1]
weights3 = [sy.Rational(-1,6), sy.Rational(5,6), sy.Rational(1,3)]
mu, eta3 = muEta(indicies3, weights3)

# Quasi-quintic
indicies5 = [-2,-1,0,1,2]
weights5 = [sy.Rational(1,30), sy.Rational(-13,60), sy.Rational(47/60), sy.Rational(9/20), 
           sy.Rational(-1/20)]
mu, eta5 = muEta(indicies5, weights5)

# Quasi-quartic
indicies4 = [-2,-1,0,1]
weights4 = [sy.Rational(1,12), sy.Rational(-5/12), sy.Rational(13/12), sy.Rational(1/4)]
mu, eta4 = muEta(indicies4, weights4)

In [ ]:
# RK3, RK4 and RK1 Butcher Tableau
quarter = sy.Rational(1,4)
sixth = sy.Rational(1,6)
half  = sy.Rational(1,2)
third = sy.Rational(1,3)
RK3 = [[1,0,0], [quarter,quarter,0], [sixth, sixth, 4*sixth]]
RK4 = [[half,0,0,0], [0, half,0,0], [0,0,1,0], [sixth, third, third, sixth]]
RK1 = [[1]]

In [ ]:
# Amplification factors for Strang carry-over AdImEx for a given RK scheme
# and a given spatial discretistaion (defined by mu and eta)
# as a function of Courant number, c and parameters alpha, beta and gamma
c, alpha, beta, gamma = sy.symbols("c, alpha, beta, gamma", real=True, positive=True)
def ampFactor(RK, mu, eta):
    A = sy.Matrix(sy.symarray("A", (len(RK)+2,)))
    A[0] = 1 - c*(1-alpha)*beta*mu
    for i in range(1,len(RK)+1):
        A[i] = A[0]
        for j in range(0,i):
            A[i] -= c*((1-beta)*mu + gamma*eta)*RK[i-1][j]*A[j]
    A[-1] = A[-2]/(1 + c*alpha*beta*mu)
    return A[-1]

In [ ]:
# Amplification factors for specific schemes
A33 = ampFactor(RK3, mu, eta3)
A34 = ampFactor(RK4, mu, eta3)
A53 = ampFactor(RK3, mu, eta5)
A54 = ampFactor(RK4, mu, eta5)
A43 = ampFactor(RK3, mu, eta4)
A44 = ampFactor(RK4, mu, eta4)

In [ ]:
# Create a matrix of maximum amplification factors for a given scheme, for 
# a range of Courant numbers, cs, and a range of gammas, gs, over a range of kDxs
def maxMagA(cs, gs, A):
    kdxs = np.linspace(np.pi/4, np.pi, 10)
    magA = np.zeros([len(gs), len(cs)])
    for ic in range(len(cs)):
        cTmp = cs[ic]
        aTmp = 1-1/max(2,cTmp)
        bTmp = 1-1/max(1,cTmp)
        for ig in range(len(gs)):
            for ik in range(len(kdxs)):
                Atmp = A.subs({alpha: aTmp, beta: bTmp, kDx: kdxs[ik], c: cTmp, gamma: gs[ig]})
                magA[ig,ic] = max(magA[ig,ic], abs(Atmp.evalf()))
    return magA

In [ ]:
cs = 0.1*10**np.linspace(0,3,13)
gs = (0.001)*10**np.linspace(0,3,13)

In [ ]:
magA33 = maxMagA(cs, gs, A33)
magA34 = maxMagA(cs, gs, A34)
magA53 = maxMagA(cs, gs, A53)
magA54 = maxMagA(cs, gs, A54)
magA43 = maxMagA(cs, gs, A43)
magA44 = maxMagA(cs, gs, A44)

In [ ]:
def gammaPoly(c, cMax, grad):
    a = (2/grad)**2
    b = cMax - 2/grad
    return a/(c-b)**2

def gammaLinear(c, cMax, grad):
    a = 1/grad
    b = cMax - a
    return a/(c-b)

In [ ]:
# Plot of magA33
magA = magA33
fig, ax = plt.subplots()
ax1 = ax.contourf(cs,gs, magA, levels=np.linspace(0,2,21), extend='max', cmap='seismic')
fig.colorbar(ax1)
ax.set_xscale('log')
ax.set_yscale('log')
ax.contour(cs, gs, magA, [-1,1], colors='k')
ax.plot(cs, gammaLinear(cs, 2, 0.15), 'c--', label='lin 2, 0.15')
ax.plot(cs, gammaLinear(cs, 2, 0.2), 'm--', label='lin 2, 0.2')
ax.legend()
ax.set_xlabel('c')
ax.set_ylabel(r'$\gamma$')
ax.set_ylim([0.1,1])
ax.set_xlim([1, 30])
plt.show()

In [ ]:
# Plot of magA34
magA = magA34
fig, ax = plt.subplots()
ax1 = ax.contourf(cs,gs, magA, levels=np.linspace(0,2,21), extend='max', cmap='seismic')
fig.colorbar(ax1)
ax.set_xscale('log')
ax.set_yscale('log')
ax.contour(cs, gs, magA, [-1,1], colors='k')
ax.plot(cs, gammaLinear(cs, 3, 0.25), 'c--', label='lin 3, 1/4')
ax.plot(cs, gammaLinear(cs, 3, 0.2),  'm--', label='lin 3, 0.2')
ax.legend()
ax.set_xlabel('c')
ax.set_ylabel(r'$\gamma$')
ax.set_ylim([0.1,1])
ax.set_xlim([1, 30])
plt.show()

In [ ]:
# Plot of magA53
magA = magA53
fig, ax = plt.subplots()
ax1 = ax.contourf(cs,gs, magA, levels=np.linspace(0,2,21), extend='max', cmap='seismic')
fig.colorbar(ax1)
ax.set_xscale('log')
ax.set_yscale('log')
ax.contour(cs, gs, magA, [-1,1], colors='k')
ax.plot(cs, gammaLinear(cs, 1, 0.25), 'b--', label='lin 1, 1/4')
ax.plot(cs, gammaLinear(cs, 1, 0.3), 'r--', label='lin 1, 0.3')
ax.legend()
ax.set_xlabel('c')
ax.set_ylabel(r'$\gamma$')
ax.set_ylim([0.1,1])
ax.set_xlim([0.8, 10])
plt.show()

In [ ]:
# Plot of magA54
magA = magA54
fig, ax = plt.subplots()
ax1 = ax.contourf(cs,gs, magA, levels=np.linspace(0,2,21), extend='max', cmap='seismic')
fig.colorbar(ax1)
ax.set_xscale('log')
ax.set_yscale('log')
ax.contour(cs, gs, magA, [-1,1], colors='k')
ax.plot(cs, gammaLinear(cs, 1.4, 0.4), 'c--', label='lin 1.4, 0.4')
ax.plot(cs, gammaLinear(cs, 1.4, 0.3), 'm--', label='lin 1.4, 0.3')
ax.legend()
ax.set_xlabel('c')
ax.set_ylabel(r'$\gamma$')
ax.set_ylim([0.1,1])
ax.set_xlim([1, 30])
plt.show()

In [ ]:
# Plot of magA44
magA = magA44
fig, ax = plt.subplots()
ax1 = ax.contourf(cs,gs, magA, levels=np.linspace(0,2,21), extend='max', cmap='seismic')
fig.colorbar(ax1)
ax.set_xscale('log')
ax.set_yscale('log')
ax.contour(cs, gs, magA, [-1,1], colors='k')
ax.plot(cs, gammaLinear(cs, 1.7, 0.4), 'm--', label='lin 1.7, 0.4')
ax.plot(cs, gammaLinear(cs, 1.7, 0.31), 'c--', label='lin 1.7, 0.31')
ax.legend()
ax.set_xlabel('c')
ax.set_ylabel(r'$\gamma$')
ax.set_ylim([0.1,1])
ax.set_xlim([1, 30])
plt.show()

In [ ]:
# Plot of magA43
magA = magA43
fig, ax = plt.subplots()
ax1 = ax.contourf(cs,gs, magA, levels=np.linspace(0,2,21), extend='max', cmap='seismic')
fig.colorbar(ax1)
ax.set_xscale('log')
ax.set_yscale('log')
ax.contour(cs, gs, magA, [-1,1], colors='k')
ax.plot(cs, gammaLinear(cs, 1.7, 0.4), 'm--', label='lin 1.7, 0.4')
ax.plot(cs, gammaLinear(cs, 1.7, 0.31), 'c--', label='lin 1.7, 0.31')
ax.legend()
ax.set_xlabel('c')
ax.set_ylabel(r'$\gamma$')
ax.set_ylim([0.1,1])
ax.set_xlim([cs[0], 30])
plt.show()